In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
from pypdf import PdfReader, PdfWriter
import pandas as pd
from io import StringIO

# Load API key from .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [2]:
def remove_pdf_password(input_path: str, password: str) -> str:
    try:
        # Prepare output file path in same folder
        base_dir = os.path.dirname(input_path)
        base_name = os.path.splitext(os.path.basename(input_path))[0]
        output_path = os.path.join(base_dir, f"{base_name}_decrypted.pdf")

        reader = PdfReader(input_path)

        if reader.is_encrypted:
            if not reader.decrypt(password):
                return "❌ Incorrect password. Cannot decrypt PDF."

        writer = PdfWriter()
        for page in reader.pages:
            writer.add_page(page)

        with open(output_path, "wb") as f:
            writer.write(f)

        print(f"✅ Password removed successfully. Saved to {output_path}")
        return output_path

    except Exception as e:
        return f"❌ Error: {str(e)}"


In [3]:
def extract_text_from_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + "\n"
    return text

def ask_gpt_for_portfolio_table(pdf_text: str) -> str:
    prompt = (
        "The following is the extracted text from a Consolidated Account Statement (CAS) "
        "of mutual funds. Extract and format the mutual fund portfolio in a table format "
        "with these columns: Folio No, Scheme Name, Unit Balance, NAV, NAV Date, Registrar, ISIN, Cost Value, Market Value.\n\n"
        f"Text:\n{pdf_text}\n\n"
        "Return only the table."
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content


In [4]:
input_path = "data/CAS_19042025-18072025_CP190329010_18072025074057751.pdf"
pdf_wo_pwd_path = remove_pdf_password(input_path, "Sidhu@0321")

# === Run the script ===
# pdf_path = "data/CAS_19042025-18072025_CP190329010_18072025074057751_decrypted.pdf"
pdf_text = extract_text_from_pdf(pdf_wo_pwd_path)
table = ask_gpt_for_portfolio_table(pdf_text)

print("\n📊 Extracted Mutual Fund Portfolio:\n")
print(table)

✅ Password removed successfully. Saved to data\CAS_19042025-18072025_CP190329010_18072025074057751_decrypted.pdf

📊 Extracted Mutual Fund Portfolio:

| Folio No         | Scheme Name                                                                 | Unit Balance | NAV Date  | NAV      | Registrar | ISIN           | Cost Value | Market Value |
|------------------|-----------------------------------------------------------------------------|--------------|-----------|----------|-----------|----------------|------------|--------------|
| 1046576943      | B66Y - Aditya Birla Sun Life PSU Equity Fund Direct-Growth (Demat )         | 2,714.049    | 17-Jul-2025 | 36.62   | CAMS     | INF209KB1O82   | 103,000.000 | 99,388.47   |
| 5057785/64      | GD340 - Bandhan Small Cap Fund-Direct Plan-Growth (Demat )                  | 2,150.060    | 17-Jul-2025 | 52.783  | CAMS     | INF194KB1AL4   | 105,000.000 | 113,486.62  |
| 32627034/46     | P5005 - ICICI Prudential BHARAT 22 FOF - Direct Plan (De

In [5]:
def clean_portfolio_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # Remove commas and convert to float
    for col in ["Unit Balance", "Cost Value", "Market Value"]:
        df[col] = df[col].str.replace(",", "").astype(float)

    # Convert NAV if not already float
    if df["NAV"].dtype != float:
        df["NAV"] = df["NAV"].astype(float)

    # Convert NAV Date to datetime
    df["NAV Date"] = pd.to_datetime(df["NAV Date"], format="%d-%b-%Y")

    # Ensure other columns are strings
    for col in ["Scheme Name", "Registrar", "ISIN", "Folio No"]:
        df[col] = df[col].astype(str).str.strip()

    return df


In [6]:
def markdown_table_to_dataframe(table_str: str) -> pd.DataFrame:
    # Remove leading/trailing spaces and clean up repeated header lines if needed
    lines = table_str.strip().split('\n')

    # Filter out separator line (with dashes)
    lines = [line for line in lines if not set(line.strip()) <= {'|', '-', ' '}]

    # Join cleaned lines into CSV-like format
    clean_table = '\n'.join(lines)
    
    # Convert to DataFrame
    df = pd.read_csv(StringIO(clean_table), sep='|', engine='python')
    
    # Remove unnamed columns from empty edges (if any)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
    
    # Strip spaces
    df.columns = df.columns.str.strip()
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].str.strip()
    
    # Clean the DataFrame
    df = clean_portfolio_dataframe(df)

    return df


In [7]:
df = markdown_table_to_dataframe(table)
df.to_csv("data/portfolio_summary.csv", index=False)


In [8]:
df

,Folio No,Scheme Name,Unit Balance,NAV Date,NAV,Registrar,ISIN,Cost Value,Market Value
0,1046576943,B66Y - Aditya Birla Sun Life PSU Equity Fund D...,2714.049,2025-07-17,36.6200,CAMS,INF209KB1O82,103000.00,99388.47
1,5057785/64,GD340 - Bandhan Small Cap Fund-Direct Plan-Gro...,2150.060,2025-07-17,52.7830,CAMS,INF194KB1AL4,105000.00,113486.62
2,32627034/46,P5005 - ICICI Prudential BHARAT 22 FOF - Direc...,7787.640,2025-07-17,32.1584,CAMS,INF109KC1FX1,247027.05,250438.04
3,32627034/46,P8184 - ICICI Prudential Infrastructure Fund -...,232.914,2025-07-17,216.4000,CAMS,INF109K018M4,49000.00,50402.59
4,31024152732/0,120ISD1G - Invesco India Infrastructure Fund -...,1431.054,2025-07-17,79.0300,KFINTECH,INF205K01MD4,107000.00,113096.20
5,31024152732/0,120MCD 1G - Invesco India Midcap Fund - Direct...,115.283,2025-07-17,220.3300,KFINTECH,INF205K01MV6,25000.00,25400.30
6,31024152732/0,120SCD1G - Invesco India Small Cap Fund - Dire...,104.292,2025-07-17,47.9400,KFINTECH,INF205K013T3,5000.00,4999.76
7,91070645216/0,127LCGDG - Motilal Oswal Large Cap Fund - Dire...,1741.672,2025-07-17,14.3575,KFINTECH,INF247L01CE3,25000.00,25006.06
8,91070645216/0,127SCGDG - Motilal Oswal Small Cap Fund - Dire...,1382.481,2025-07-17,14.6415,KFINTECH,INF247L01BY3,20000.00,20241.60
9,91070645216/0,127FMGDG - Motilal Oswal Midcap Fund - Direct ...,1195.330,2025-07-17,118.4243,KFINTECH,INF247L01445,138000.00,141556.12


In [9]:
df.dtypes

Folio No                object
Scheme Name             object
Unit Balance           float64
NAV Date        datetime64[ns]
NAV                    float64
Registrar               object
ISIN                    object
Cost Value             float64
Market Value           float64
dtype: object

In [10]:
df['Market Value'].sum()

np.float64(1589095.4900000002)

In [11]:
df["Scheme Name"] = df["Scheme Name"].str.replace(r"^[\w\s]+ -\s*", "", regex=True)
df["Scheme Name"] = df["Scheme Name"].str.replace(r"\s*\(.*?\)", "", regex=True)

In [12]:
df.head()

,Folio No,Scheme Name,Unit Balance,NAV Date,NAV,Registrar,ISIN,Cost Value,Market Value
0,1046576943,Aditya Birla Sun Life PSU Equity Fund Direct-G...,2714.049,2025-07-17,36.6200,CAMS,INF209KB1O82,103000.00,99388.47
1,5057785/64,Bandhan Small Cap Fund-Direct Plan-Growth,2150.060,2025-07-17,52.7830,CAMS,INF194KB1AL4,105000.00,113486.62
2,32627034/46,ICICI Prudential BHARAT 22 FOF - Direct Plan,7787.640,2025-07-17,32.1584,CAMS,INF109KC1FX1,247027.05,250438.04
3,32627034/46,ICICI Prudential Infrastructure Fund - Direct ...,232.914,2025-07-17,216.4000,CAMS,INF109K018M4,49000.00,50402.59
4,31024152732/0,Invesco India Infrastructure Fund - Direct Pla...,1431.054,2025-07-17,79.0300,KFINTECH,INF205K01MD4,107000.00,113096.20


In [16]:
df['Scheme Name']

0     Aditya Birla Sun Life PSU Equity Fund Direct-G...
1             Bandhan Small Cap Fund-Direct Plan-Growth
2          ICICI Prudential BHARAT 22 FOF - Direct Plan
3     ICICI Prudential Infrastructure Fund - Direct ...
4     Invesco India Infrastructure Fund - Direct Pla...
5        Invesco India Midcap Fund - Direct Plan Growth
6     Invesco India Small Cap Fund - Direct Plan Growth
7     Motilal Oswal Large Cap Fund - Direct Plan Growth
8     Motilal Oswal Small Cap Fund - Direct Plan Growth
9        Motilal Oswal Midcap Fund - Direct Plan Growth
10    NIPPON INDIA POWER & INFRA FUND - DIRECT PLAN ...
11              quant ELSS Tax Saver Fund - Direct Plan
12       quant Infrastructure Fund - Direct Plan Growth
13            quant Small Cap Fund - Direct Plan Growth
14                 SBI Gold Fund - Direct Plan - Growth
15                  SBI PSU Fund - Direct Plan - Growth
Name: Scheme Name, dtype: object